In [9]:
import os
import xml.etree.ElementTree as ET
import kagglehub

# Download latest version
path = kagglehub.dataset_download("stanislavlevendeev/hazmat-detection")

print("Path to dataset files:", path)


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


100%|██████████| 14.7M/14.7M [00:01<00:00, 8.08MB/s]

Extracting files...


Path to dataset files: C:\Users\ewald\.cache\kagglehub\datasets\stanislavlevendeev\hazmat-detection\versions\1


In [30]:
# Pad naar het XML-bestand
file_path = os.path.join(path, 'cvat-for-video-dataset-hazmat-codes', 'annotations.xml')

# XML-bestand openen en parsen
tree = ET.parse(file_path)
root = tree.getroot()

# Taken ophalen
tasks = []
for task in root.findall(".//task"):
    task_id = task.find("id").text
    task_source = task.find("source").text
    tasks.append({"id": task_id, "source": task_source})

# Taken printen
for task in tasks:
    print(f"Task ID: {task['id']}, source: {task['source']}")

print(len(tasks), "tasks found")

Task ID: 138, source: 1690279852.mp4
Task ID: 142, source: 1690281365.mp4
Task ID: 143, source: 1690279852.mp4
Task ID: 148, source: 1692830440.mp4
Task ID: 149, source: 1690801380.mp4
Task ID: 150, source: 1691487366.mp4
Task ID: 151, source: 1692787289.mp4
Task ID: 155, source: 1692875102.mp4
Task ID: 158, source: 1692945482.mp4
Task ID: 161, source: 1693811855.mp4
Task ID: 162, source: 1693954819.mp4
Task ID: 163, source: 1693805101.mp4
Task ID: 164, source: 1693820172.mp4
Task ID: 166, source: 1692787289.mp4
Task ID: 167, source: 1691496786.mp4
Task ID: 168, source: 1692872075.mp4
Task ID: 169, source: 1693308534.mp4
Task ID: 170, source: 1693308657.mp4
Task ID: 171, source: 1693309263.mp4
Task ID: 172, source: 1693820241.mp4
Task ID: 173, source: 1693820504.mp4
Task ID: 174, source: 1693820904.mp4
Task ID: 175, source: 1696009577.mp4
Task ID: 176, source: 1696374314.mp4
Task ID: 177, source: 1696416413.mp4
Task ID: 178, source: 1696441496.mp4
26 tasks found


In [31]:
# Directory containing video files
video_directory = os.environ["PATH_TO_DATA"]
print (video_directory)

data/videos


In [36]:
video_files = []
#  make a list of only the videos mentioned in the tasks
for task in tasks:
    video_name = task["source"]
    video_path = os.path.join(video_directory, video_name)
    if os.path.exists(video_path):
        video_files.append(video_path)
    else:
        print(f"Video file {video_path} not found")
print(f"{len(video_files)}/{len(tasks)} video files found")

Video file data/videos\1692830440.mp4 not found
Video file data/videos\1690801380.mp4 not found
Video file data/videos\1691487366.mp4 not found
Video file data/videos\1692787289.mp4 not found
Video file data/videos\1692945482.mp4 not found
Video file data/videos\1693811855.mp4 not found
Video file data/videos\1693954819.mp4 not found
Video file data/videos\1693805101.mp4 not found
Video file data/videos\1693820172.mp4 not found
Video file data/videos\1692787289.mp4 not found
Video file data/videos\1691496786.mp4 not found
Video file data/videos\1692872075.mp4 not found
Video file data/videos\1693308657.mp4 not found
Video file data/videos\1693309263.mp4 not found
Video file data/videos\1693820241.mp4 not found
Video file data/videos\1693820504.mp4 not found
Video file data/videos\1693820904.mp4 not found
Video file data/videos\1696009577.mp4 not found
Video file data/videos\1696374314.mp4 not found
Video file data/videos\1696416413.mp4 not found
Video file data/videos\1696441496.mp4 no